# Osa 7: Foreign Data Wrapper (fdw)
Tässä osassa tutustutaan fdw:ihin csv-tiedoston lukemisen kautta.

In [ ]:
# Lataa SQL-lisäosa
%load_ext sql
# Luo tietokantayhteys tietokantaan koulutus
from ipygis import get_connection_url
c_url = get_connection_url(dbname='koulutus')
%sql $c_url

## file_fdw -lisäosa tiedostojen lukemiseen
Asenna aluksi `file_fdw`-lisäosa, jonka kautta voidaan luoda fdw csv-tiedostojen lukemiseen.

In [ ]:
%sql CREATE EXTENSION file_fdw;

Luo sitten `foreign server` käyttäen file_fdw fdw:tä.

In [ ]:
%sql CREATE SERVER log_server FOREIGN DATA WRAPPER file_fdw;

Ladataan nyt vieraana tauluna sama [etunimet.csv](/edit/data/etunimet.csv)-tiedosto, kuin aikaisemmin tehdyssä [harjoituksessa](H3_tietokannat_ja_taulut.ipynb#Harjoitus-3.2):

In [ ]:
%%sql 

CREATE FOREIGN TABLE etunimet2 (
    nimi text NOT NULL,
    lukumaara integer NOT NULL,
    sukupuoli text NOT NULL
) SERVER log_server OPTIONS(filename '/home/oppilas/pg-training/Harjoitukset/data/etunimet.csv', 
                            format 'csv', header 'true');

Tarkistetaan, että taulusta lukeminen toimii:

In [ ]:
%sql SELECT * FROM etunimet2 LIMIT 20;

Vieraan taulun voi poistaa syntaksilla:

```sql
DROP FOREIGN TABLE taulun_nimi;
```

In [ ]:
%sql DROP FOREIGN TABLE etunimet2;

## **Harjoitus 7.1**: csv-tiedostosta lukeminen
Luo vieras taulu `audit_log` käyttäen tiedostoa [audit_log.csv](/edit/data/audit_log.csv). Kenttien nimet ovat ensimmäisellä rivillä.

In [ ]:
%%sql

FOREIGN TABLE audit_log (
    log_time timestamp with time zone,
    user_name text,
    database_name text,
    process_id integer,
    connection_from text,
    session_id text,
    session_line_num integer,
    process_status text,
    session_start_time timestamp with time zone,
    virtual_transaction_id text,
    transaction_id integer,
    error_severity text,
    sql_state_code integer,
    message text,
    detail text,
    hint text,
    internal_query text,
    internal_query_pos text,
    context text,
    query text,
    query_pos integer,
    location text,
    application_name text,
    command_tag text,
    audit_tag text
) SERVER ...

In [ ]:
%%sql

CREATE FOREIGN TABLE audit_log (
    log_time timestamp with time zone,
    user_name text,
    database_name text,
    process_id integer,
    connection_from text,
    session_id text,
    session_line_num integer,
    process_status text,
    session_start_time timestamp with time zone,
    virtual_transaction_id text,
    transaction_id integer,
    error_severity text,
    sql_state_code integer,
    message text,
    detail text,
    hint text,
    internal_query text,
    internal_query_pos text,
    context text,
    query text,
    query_pos integer,
    location text,
    application_name text,
    command_tag text,
    audit_tag text
) SERVER log_server OPTIONS(filename '/home/oppilas/pg-training/Harjoitukset/data/audit_log.csv', 
                            format 'csv', header 'true');

Listaa `audit_log`-taulun rivit

In [ ]:
res = %sql SELECT * FROM audit_log;
res

### Tarkistus
Aja alla oleva solu tarkistaaksesi, että rivejä on oikea määrä

In [ ]:
assert len(res) == 32, f'Rivejä ei ole oikea määrä: {len(res)}'

## **Harjoitus 7.2:** Toisesta PostgreSQL-kannasta lukeminen

Tässä harjoituksessa luetaan tietokannan `postgres` taulua `kunnat` kannan `koulutus` sisältä.
Ota aluksi käyttöön lisäosa `postgres_fdw`



In [ ]:
%sql CREATE ...

In [ ]:
%sql CREATE EXTENSION postgres_fdw;

Luo sitten foreign server `postgres_server` käyttäen postgres_fdw:tä

In [ ]:
%%sql
CREATE SERVER postgres_server
        FOREIGN DATA WRAPPER postgres_fdw
        OPTIONS (host 'localhost', port '5432', dbname 'postgres');

postgres_fdw:n kanssa on tehtävä myös käyttäjämäppäys, jolla nykyisen tietokannan käyttäjälle annetaan vastine toisen tietokannan parissa.

In [ ]:
%%sql
CREATE USER MAPPING FOR postgres
        SERVER postgres_server
        OPTIONS (user 'postgres', password '<KYSY SALASANA>');

Koska luemme taulua `kunnat`, ota käyttöön nykyiseen tietokantaan lisäosa `postgis`.

In [ ]:
%sql CREATE ...

In [ ]:
%sql CREATE EXTENSION postgis;

Luodaan nyt vieras taulu `kunnat`. Täytä arvot parametreihin `schema_name` ja `table_name`.

In [ ]:
%%sql
CREATE FOREIGN TABLE kunnat (
    ogc_fid integer,
    geom geometry(MultiPolygon,3067),
    nimi character varying,
    id numeric(10,0)
)
    SERVER postgres_server
    OPTIONS (schema_name 'täydennä', table_name 'täydennä');

In [ ]:
%%sql
CREATE FOREIGN TABLE kunnat (
    ogc_fid integer,
    geom geometry(MultiPolygon,3067),
    nimi character varying,
    id numeric(10,0)
)
    SERVER postgres_server
    OPTIONS (schema_name 'public', table_name 'kunnat');

Listaa kunnat-taulun rivejä

In [ ]:
res = %sql SELECT id, nimi FROM kunnat;
res

### Tarkistus
Aja alla oleva solu tarkistaaksesi, että rivejä on oikea määrä

In [ ]:
assert len(res) == 95, f'Rivejä ei ole oikea määrä: {len(res)}'

<a rel="license" href="http://creativecommons.org/licenses/by-nd/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nd/4.0/88x31.png" />